## Testing changes to the **current** repository using existing CMIP infrastructure

In this notebook we take the JSON-LD files in the current directory, copy them to a locally hosted temporary directory and load them as if they were coming from the published site (`<owner>.github.io/<repo>`)

This method allows us to test any new changes with the existing JSON-LD universe.

In [4]:
import cmipld
from cmipld.utils.offline import LD_server, replace_loader

##### Set up the temporary directory

In [5]:


# the path of the JSON-LD content for the current repository
ldpath = cmipld.utils.git.ldpath()

# create a temporary directory containing (a copy of) the current repository
localserver = LD_server(copy = [[ldpath,'current_repo']])

# NOTE: all changes to the files will need to be updated e.g.: 
# localserver.copy_existing_repos([['pat/to/repo','repo_name']...])


Copying the repo into LocalServer  /Users/daniel.ellis/WIPwork/CMIP6Plus_CVs/src-data/ --> current_repo
Repositories copied into <TemporaryDirectory '/var/folders/hc/s_7lggq12nndglbdyrn3f91m1l58yd/T/cmipld_local_k8kz8epe2025-01-15T11:46:44'>


#### Start the server

In [6]:
# After establishing a temporary directory we can start the server
# this will host all the files we have loaded on localserver:port/
localhost = localserver.start_server(8081)


...........+...+.....+.+...+..+++++++++++++++++++++++++++++++++++++++++++++*.....+..........+......+.....+.+..................+..+++++++++++++++++++++++++++++++++++++++++++++*.............+..+.+..........................+......+........................+.........+.+...+...........+...............+.+........+.+..+.......+........+.......+..+....+.................+.......+...............+.....+.+......+.....................+..+.+.....+....+.....+.......+.........+.....+....+...+.....+.............+..+...+.+........+...+...+.......+...+......+.....+.+...+..+.........+...+............+....+.....+.......+...+..+....+...+............+...........+..........+........+.........+...+...+.+...+............+.....+.+....................+.......+.....+.+..+++++
.........+.+..+.........+.+.....+....+...+..+....+..+.......+.....+.+..+.........+...+...+...+.......+..+...+.......+...+........+......+.+.........+...+.....+..........+..+.........+....+......+...............+..+..........+..+.+..+....+.....

Created SSL certificates in: /var/folders/hc/s_7lggq12nndglbdyrn3f91m1l58yd/T/cmipld_local_k8kz8epe2025-01-15T11:46:44
Serving /var/folders/hc/s_7lggq12nndglbdyrn3f91m1l58yd/T/cmipld_local_k8kz8epe2025-01-15T11:46:44 at https://localhost:8081


.........+.........+.................................................+..+....+...........+.+.................+...+......................+........+...................+...+...+...............+.........+......+.........+.....+...+..........+..+.+...+.....+....+......+..+....+...........+.......+.....+..........+...+........+.......+..+...+....+.....+..........+...........+.......+......+.........+........+...+.......+...............+...+..............+.......+...........+...+.......+........+...................+..+.............+...+........+......+.+..+......+......+++++
-----


##### Set up redirects

In [7]:

# Obtaining the hosted github io address of the current repository
repo_url = cmipld.utils.git.url()
io_url = cmipld.utils.git.url2io(repo_url)

# We now generate create a list of urls to be redirected, and replace the JSONLD loader. 
replace_loader(localhost,[(io_url,'current_repo')] )


make test for url2io
Setting up location forwarding for:
 -  https://wcrp-cmip.github.io/CMIP6Plus_CVs/ >>> https://localhost:8081/current_repo/



##### Getting the data

In [8]:
# This means we can make 'locally hosted' content appear as an external resource in our PYLD instance

testfile = f'{io_url}project/product.json'

from pyld import jsonld
jsonld.expand(testfile)

127.0.0.1 - - [15/Jan/2025 11:46:45] "GET /current_repo/project/product.json HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2025 11:46:45] "GET /current_repo/project/_context_ HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2025 11:46:45] "GET /current_repo/_context_ HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2025 11:46:50] "GET /current_repo/project/product.json HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2025 11:46:51] "GET /current_repo/project/_context_ HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2025 11:46:51] "GET /current_repo/_context_ HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2025 11:46:51] "GET /current_repo/project/_context_ HTTP/1.1" 200 -
127.0.0.1 - - [15/Jan/2025 11:46:51] "GET /current_repo/_context_ HTTP/1.1" 200 -


[{'@id': 'https://wcrp-cmip.github.io/CMIP6Plus_CVs/project/product',
  'https://wcrp-cmip.github.io/CMIP6Plus_CVs/project/product': [{'@value': 'model-output'}],
  '@type': ['wcrp:project',
   'https://wcrp-cmip.github.io/CMIP6Plus_CVs/',
   'wcrp:product']}]

In [9]:
# or the cmipld library loaders.
cmipld.get(testfile)


[{'@context': 'https://wcrp-cmip.github.io/CMIP6Plus_CVs/project/_context_',
  'id': 'cmip6plus:project/product',
  'type': ['wcrp:project', 'cmip6plus', 'wcrp:product'],
  'cmip6plus:project/product': 'model-output'}]

In [12]:
LD_server.stop_server()

AttributeError: type object 'LD_server' has no attribute 'server'